<a href="https://colab.research.google.com/github/ramon284/MachineLearningVu/blob/main/LSTM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
#Load values created by data preprocessing


max_sequence_len = 90 #Hardcoded, taken from data build script
total_words = 6215  #Hardcoded, taken from data build script

In [25]:
import numpy

labeltemp = numpy.load("/content/drive/MyDrive/all_labels.npy")

labels = [labeltemp]

In [26]:
predictors = numpy.load("/content/drive/MyDrive/predictors.npy")


In [27]:
print(labels)
print(predictors)

[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)]
[[  0   0   0 ...   0   0   8]
 [  0   0   0 ...   0   8   9]
 [  0   0   0 ...   8   9  36]
 ...
 [  0   0   0 ... 643  13  27]
 [  0   0   0 ...  13  27  40]
 [  0   0   0 ...  27  40 323]]


In [28]:
#Check what gpu/tpu we are running on
!nvidia-smi

Thu Mar  4 14:05:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    72W / 149W |    189MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

In [30]:

# #4. Model
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 89, 10)            62150     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6215)              627715    
Total params: 734,265
Trainable params: 734,265
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Train the Model
model.fit(predictors, labels, epochs=20, verbose=1)

Epoch 1/20
1563/1563 [==============================] - 35s 22ms/step - loss: 6.5771
Epoch 2/20
1563/1563 [==============================] - 35s 22ms/step - loss: 6.2285
Epoch 3/20
1563/1563 [==============================] - 35s 22ms/step - loss: 6.0055
Epoch 4/20
1563/1563 [==============================] - 35s 22ms/step - loss: 5.7876
Epoch 5/20
 808/1563 [==============>...............] - ETA: 17s - loss: 5.5733

In [ ]:
# # #5. Generating texts
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()